In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

import time

from hist import Hist

import babar_analysis_tools as bat
from analysis_variables import *
import myPIDselector

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

import joblib

In [ ]:
print(np.__version__)

print(plt.__version__)

# Functions

In [ ]:
def my_train_test_split(df, bkg_spmode='998', sig_spmode='-999', n_sig_bkg=[1000,1000], \
                        test_size=0.4, \
                        columns_to_drop=["spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass"]):

    columns= df.columns

    print("columns...")
    print(columns)
    print()

    feature_names= columns[1:] ##exclude spmode
    #print(feature_names)
    
    filter_sig= df["spmode"]==sig_spmode
    filter_bkg= df["spmode"]==bkg_spmode

    # Get the indices of the sig and bkg
    idx_sig = df[filter_sig].index.values
    idx_bkg = df[filter_bkg].index.values

    print(idx_sig)
    
    # Shuffle them
    np.random.shuffle(idx_sig)
    np.random.shuffle(idx_bkg)

    #df_sig= df[filter_sig].dropna().sample(n_sig_bkg[0])
    #df_bkg= df[filter_bkg].dropna().sample(n_sig_bkg[1])

    # Amount to use for both training and testing
    n_sig = n_sig_bkg[0]
    n_bkg = n_sig_bkg[1]

    # Amount to use for testing
    n_test_sig = int(n_sig*test_size)
    n_train_sig = n_sig - n_test_sig

    # Amount to use for training
    n_test_bkg = int(n_bkg*test_size)
    n_train_bkg = n_bkg - n_test_bkg

    print("Training breakdown...")
    print(f"sig:   train: {n_train_sig}    test: {n_test_sig}") 
    print(f"bkg:   train: {n_train_bkg}    test: {n_test_bkg}") 
    
    #print(len(df_sig), len(df_bkg))
    
    #df_ML= pd.concat([df_sig,df_bkg])
    
    #x=df_ML.drop(columns= columns_to_drop)
    #y=df_ML["spmode"]

    idx_sig_train = idx_sig[0:n_train_sig]
    idx_sig_test  = idx_sig[n_train_sig:n_train_sig+n_test_sig]
    idx_sig_not_train = idx_sig[n_train_sig:]

    idx_bkg_train = idx_bkg[0:n_train_bkg]
    idx_bkg_test  = idx_bkg[n_train_bkg:n_train_bkg+n_test_bkg]
    idx_bkg_not_train = idx_bkg[n_train_bkg:]

    #print(idx_sig_train)
    
    x_sig_train = df.drop(columns=columns_to_drop).loc[idx_sig_train]
    x_sig_test  = df.drop(columns=columns_to_drop).loc[idx_sig_test]

    y_sig_train = df['spmode'].loc[idx_sig_train]
    y_sig_test  = df['spmode'].loc[idx_sig_test]

    
    x_bkg_train = df.drop(columns=columns_to_drop).loc[idx_bkg_train]
    x_bkg_test  = df.drop(columns=columns_to_drop).loc[idx_bkg_test]

    y_bkg_train = df['spmode'].loc[idx_bkg_train]
    y_bkg_test  = df['spmode'].loc[idx_bkg_test]

    # Merge them
    print("here----------------------------------------------------")
    #print(x_sig_train)
    #print(x_bkg_train)
    
    x_train = pd.concat([x_sig_train, x_bkg_train])
    x_test = pd.concat([x_sig_test,  x_bkg_test])

    #x_train = x_train.values#.transpose()[1:]
    #x_test = x_test.values#.transpose()[1:]

    #print(x_train)
    
    y_train = pd.concat([y_sig_train, y_bkg_train])
    y_test = pd.concat([y_sig_test,  y_bkg_test])

    print("The dataset (x) is the numbers without column names---")
    print("The variable y is truth info about the data (signal or bkg)\n")

    return x_train, x_test,y_train,  y_test, [idx_sig_train, idx_sig_not_train, idx_bkg_train, idx_bkg_not_train]


#x_train, x_test, y_train, y_test,train_test_indices = my_train_test_split(df_sp)

'''
print(type(x_train))
print(x_train)
print()
print(x_test)
print()
print(train_test_indices[1])
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)
'''
;

In [ ]:
def model_maker(df, sig_spmode="-999", bkg_spmode= "998", n_sig_bkg=[1000,1000], \
                columns_to_drop=["spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass"], \
                test_size_pct= 0.4, activator= "relu", solve_model= "adam", model_filename=None): 
            

    #x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= test_size_pct, random_state= 4)
    x_train, x_test, y_train, y_test,train_test_indices= my_train_test_split(df, sig_spmode=sig_spmode, bkg_spmode=bkg_spmode, \
                                                          columns_to_drop=columns_to_drop, \
                                                          n_sig_bkg=n_sig_bkg, \
                                                          test_size= test_size_pct)

    feature_names = x_train.columns
    print("Feature names (model_maker)....")
    print(feature_names)
    
    # LETS TRY TO NOT DO THIS
    #scaler= StandardScaler()

    #print(type(x_train))
    #print(x_train)
    # Look in "tips for practical use"
    # https://scikit-learn.org/stable/modules/neural_networks_supervised.html
    # This also makes it no longer a DataFrame but an nxm numpy array
    #x_train= scaler.fit_transform(x_train)

    #print("x_train after........")
    #print(x_train)
    
    #x_test= scaler.transform(x_test)

    #print("x_test after..........")
    #print(x_test)
    
    model = MLPClassifier(max_iter= 1000, random_state= 3, activation= activator, solver= solve_model ) #n_iter_no_change= 15)

    model.fit(x_train, y_train)

    model.feature_names = feature_names
    
    workspace = {}
    workspace['model'] = model
    workspace['x_train'] = x_train
    workspace['y_train'] = y_train
    workspace['x_test'] = x_test
    workspace['y_test'] = y_test
    #[idx_sig_train, idx_sig_not_train, idx_bkg_train, idx_bkg_not_train]

    workspace['idx_sig_train'] = train_test_indices[0]
    workspace['idx_sig_not_train'] = train_test_indices[1]
    workspace['idx_bkg_train'] = train_test_indices[2]
    workspace['idx_bkg_not_train'] = train_test_indices[3]

    if model_filename is not None:
        joblib.dump(workspace, model_filename)

    return workspace

    

In [ ]:
def plot_training_variables(df):

    #########################################################################
    # Plot the variables for the different spmodes
    #########################################################################

    print("Plotting the training variables...")
    nvars = len(df.columns)

    nrows = 5
    ncols = int(nvars / nrows) + 1
    
    fig, axes = plt.subplots(nrows = nrows, ncols = ncols)    # axes is 2d array (3x3)
    axes = axes.flatten()         # Convert axes to 1d array of length 9
    fig.set_size_inches(ncols*3, nrows*3)

    for ax, col in zip(axes, df.columns):
      sns.histplot(df, x=col, ax = ax, hue='spmode', stat='density', common_norm=False)
      ax.set_title(col)

    plt.tight_layout()

    #########################################################################
    # Plot the correlation matrices
    #########################################################################

    spmodes_in_file = df['spmode'].unique()
    # Drop the cuts columns
    cols = df_temp.columns

    cols_temp = []
    for col in cols:
        #print(col)
        if col[0:3]!='cut':
            cols_temp.append(col)
    cols_temp

    for spmode in spmodes_in_file:
        print(f"Making the correlation matrix for SP-{spmode}...")
        fig,ax = plt.subplots(figsize=(16,16))
        mask = df_temp['spmode'] == spmode
        
        sns.heatmap(df_temp[mask][cols_temp].drop(columns=['spmode']).corr(), center=0, cmap='coolwarm', annot=True, fmt='.2f', annot_kws={"size": 8})
        plt.title(f'Correlation matrix SP {spmode}')        

In [ ]:
# 
#def model_training_quality(model, x_train, y_train, x_test, y_test):
def model_training_quality(workspace):
    model = workspace['model']
    x_train = workspace['x_train']
    y_train = workspace['y_train']
    x_test = workspace['x_test']
    y_test = workspace['y_test']
    
    #model
    ###################################################################
    # Get the predictions for the training and testing samples
    ###################################################################
    decisions = []
    for X, y in ((x_train, y_train), (x_test, y_test)):

      # Use the outcome to select the truth information (>0.5 or <0.5)
      d1 = model.predict_proba(X[y == '998'])[:, 1]
      d2 = model.predict_proba(X[y == '-999'])[:, 1]
      decisions += [d1, d2]
    
    # Use this for the histogram ranges
    low = min(np.min(d) for d in decisions)
    high = max(np.max(d) for d in decisions)
    low_high = (low, high)
    
    
    #print(decisions)
    ###################################################################
    # Make a plot of the training sample predictions
    ###################################################################

    bins = 50
    plt.figure(figsize=(12, 6))
    plt.hist(decisions[0],
              color='r', alpha=0.5, range=low_high, bins=bins,
              histtype='stepfilled', density=True,
              label='Bkg (train)')
    plt.hist(decisions[1],
              color='b', alpha=0.5, range=low_high, bins=bins,
              histtype='stepfilled', density=True,
              label='Sig (train)')
    
    
    # Make a plot with error bars for the testing samples
    hists, bins = np.histogram(decisions[2],density=True,
                              bins=bins, range=low_high)
    scale = len(decisions[2]) / sum(hists)
    err = np.sqrt(hists * scale) / scale
    
    width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    
    plt.errorbar(center, hists, yerr=err, fmt='o', c='r', label='Bkg (test)')
    
    hists, bins = np.histogram(decisions[3],density=True,
                              bins=bins, range=low_high)
    scale = len(decisions[2]) / sum(hists)
    err = np.sqrt(hists * scale) / scale
    
    plt.errorbar(center, hists, yerr=err, fmt='o', c='b', label='Sig (test)')
    
    plt.xlabel("Classifer output")
    plt.ylabel("Arbitrary units")
    plt.legend(loc='best')

    # Confusion matrix
    # Testing the model i.e. predicting the labels of the test data.
    y_pred = model.predict(x_test)
    
    # Evaluating the results of the model
    accuracy = accuracy_score(y_test,y_pred)*100 ### returns the fraction of correctly classified samples 
    confusion_mat = confusion_matrix(y_test,y_pred)

    print("Accuracy for Neural Network is:",accuracy)
    print("Confusion Matrix")
    print(confusion_mat)
    
    tot_correct= confusion_mat[1][1] + confusion_mat[0][0]
    tot_wrong= confusion_mat[1][0] + confusion_mat[0][1]
    
    #print(tot_correct/(tot_correct+tot_wrong))
    
    ## The accuracy score is the total number classified correctly over the total number of classifications 


    # Turn this into a dataframe
    matrix_df = pd.DataFrame(confusion_mat)
    
    # Plot the result
    fig, ax = plt.subplots(figsize=(10,7))
    
    sns.set(font_scale=1.3)
    
    sns.heatmap(matrix_df, annot=True, fmt="g", ax=ax, cmap="magma")
    
    #labels = df['target_names'].tolist()
    labels = ['998', '-999'] # NEED TO FIX THIS SO IT IS NOT HARDCODED
    
    # Formatting details here
    # Set axis titles
    ax.set_title('Confusion Matrix - MLP')
    ax.set_xlabel("Predicted label", fontsize =15)
    ax.set_xticklabels(labels)
    ax.set_ylabel("True Label", fontsize=15)
    ax.set_yticklabels(labels, rotation = 0)
    #plt.show()

    # ROC

    decisions = model.predict_proba(x_test)[:, 1]

    ###################################################################
    # Compute ROC curve and area under the curve
    ###################################################################

    sig_bkg = np.ones_like(y_test, dtype=int)
    sig_bkg[y_test=='-999'] = 0

    fpr, tpr, thresholds = roc_curve(sig_bkg, decisions)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, lw=1, label='ROC (area = %0.2f)' % (roc_auc))
    
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()


In [ ]:
#""""
def add_probas_to_dfs(workspace, df_col, df_sp):

    model = workspace['model']
    x_train = workspace['x_train']
    y_train = workspace['y_train']
    x_test = workspace['x_test']
    y_test = workspace['y_test']

    #idx_x_train     = workspace['idx_sig_train']
    #idx_x_not_train = workspace['idx_sig_not_train']

    # Get the training vars
    training_vars = model.feature_names

    print("Training vars: ")
    print(training_vars)
    print(len(training_vars))

    remove_these = ['proba', 'used_in_sig_train', 'used_in_bkg_train']
    print(type(training_vars))
    for r in remove_these:
        print(f"Checking for {r}...")
        if sum(training_vars.isin([r]).astype(int))>0:
            idx = training_vars.get_loc(r)
            print(idx, training_vars)
            training_vars = training_vars.delete(idx)
            print(training_vars)
            print(f"removed {r}")
    
    #x_train = df_col[training_vars]#[idx_x_train]

    #### FOR SP 
    #for i in threshold:
    #    output_df= see_stuff(sig_samp= sig_samp,bkg_samp= bkg_samp, thresh= i, verbose= False, df=MC_data, df_col=coll_data)

    # 3. Scale the test data using the same scaler
    #scaler = StandardScaler()

    # SHOULD THIS BE THE ACTUAL TRAINING SP
    #x_dummy = scaler.fit_transform(x_train)

    #print(x_dummy.T[-1])
    #print(len(x_dummy), len(x_dummy[0]))

    print("training vars after" )
    print(training_vars)
    # Collision
    x_test  = df_col[training_vars].values#[idx_x_not_train]
    print(x_test.T[-1])

    #x_test = scaler.transform(x_test)

    
    proba = model.predict_proba(x_test)

    print("proba: ")
    print(proba)
    
    df_col['proba'] = proba[:,0]
    
    print(df_col.columns)

    # SP
    # Collision
    x_test  = df_sp[training_vars].values#[idx_x_not_train]
    #x_test = scaler.transform(x_test)

    print(x_test)
    
    proba = model.predict_proba(x_test)

    print("proba: ")
    print(proba)
    
    df_sp['proba'] = proba[:,0]
    
    print(df_sp.columns)

    idx_bkg_train = workspace['idx_bkg_train']
    idx_sig_train = workspace['idx_sig_train']

    nentries = len(df_sp)
    df_sp['used_in_sig_train'] = np.zeros(nentries, dtype=bool)
    df_sp.loc[idx_sig_train, "used_in_sig_train"] = True

    df_sp['used_in_bkg_train'] = np.zeros(nentries, dtype=bool)
    df_sp.loc[idx_bkg_train, 'used_in_bkg_train'] = True

    
    #x_col_proba = model.predict_proba(df_col_tmp)
    
    #### FOR COLLISION
    #y_proba_col_sig = model.predict_proba()
    
    #sp998= sp_data["spmode"]== "998"
    #N_bkg = len(sp_998_df[sp998]) ## total number of background events (sp 998) 
    #signal_before= len(sp_999_df)
    #signal_after= len(sp_999_df)
    #efficiency = signal_after/signal_before ## the accuracy of the model after training with the SP 
    #fom = efficiency(threshold)/(np.sqrt(N_bkg(threshold)+sig_disc/2))
    #return fom
    return 0
#"""

# Analysis

In [ ]:
#BNC_tag = ""
#BNC_bool = False

# BNC
BNC_tag = "_BNC"
BNC_bool = True


#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
#topdir= "/home/bellis/babar_data/bnv_plambda"
topdir= "/home/bellis/babar_data_local/bnv_plambda"

if BNC_bool:
    #topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
    topdir= "/home/bellis/babar_data_local/bnv_plambda_bnc"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda"
#####################################################################

#####################################################################
# Get the BNV data
#####################################################################
#data, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')
#data, data_collision = bat.load_datasets(topdir=topdir, subset='all')

#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
data, data_collision = bat.load_datasets(topdir=topdir, BNC=BNC_bool, subset='all')
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')
#BNC_tag = "_BNC"
#BNC_bool = True

In [ ]:
#data.columns
data.fields

In [ ]:
#data["BMass"]

In [ ]:
dataset_information= pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

sp= data["spmode"]

splist= np.unique(sp.to_list())
splist

'''
dcuts= bat.get_final_masks(data, region_definitions= region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

dcuts[3]
'''

bat.fill_new_entry_with_tag_side_B(data)
data["BtagSideMes"]
bat.fill_new_entry_with_tag_side_B(data_collision)
data_collision["BtagSideMes"]

all_hists= bat.create_empty_histograms(hist_defs)

bkg_spmodes= ["998","1005","3981","1235","1237"]
sig_spmodes= ["-999"]

spmodes= bkg_spmodes+sig_spmodes

weights= {}
for sp in spmodes: 
    weights[sp]= bat.scaling_value(int(sp),dataset_information=dataset_information, cs_data= cs_data, plot= False, verbose= False)
    #weights[sp]=1

weights["-999"]= 1000 #scales signal higher 
weights["0"]= 1 #idk what this is for;;; ASK

print(weights)

In [ ]:
mask_sp_data = data['spmode'] == '-999'
len(data[mask_sp_data])

In [ ]:
'''
#mask_event= dcuts[1]["event"]
#mask_event= dcuts[2]["event"]
mask_event= dcuts[3]["event"]
#mask_event= dcuts[4]["event"] ## individual cuts
#mask_event= dcuts[-1]["event"] ## all cuts

#mask_event= dcuts[1]["event"] & dcuts[2]["event"] & dcuts[3]["event"] & dcuts[4]["event"] ## combo of cuts
mask_event= dcuts[1]["event"] & dcuts[2]["event"] & dcuts[3]["event"] ## combo of cuts
'''
### ASK WHAT THESE MEAN
#tag= "EARLY_CUT"
cuts_tag= "CUTS_cuts_1_2_3"

#mask= mask_event

In [ ]:
'''
subset = ['spmode', 'BpostFitMes', 'BpostFitDeltaE', 'Lambda0_unc_Mass', \
          'BtagSideMes', 'BSphr', 'BThrust', 'BCosThetaS', \
          'R2', 'R2All', \
          'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
          'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
          'BThrustROE']

ak_array_type= type(data["spmode"])

df_dict={}
for var in subset: 
    x= data[mask][var] ##in each event, cut on the above cuts and pull out the info from each of the variables listed above
    if type(x[0]) == ak_array_type:
        x= ak.flatten(data[mask][var])
    df_dict[var] = x

df_out= pd.DataFrame.from_dict(df_dict)

outfilename= f"output_variables_{tag}.parquet"
df_out.to_parquet(outfilename)

df= df_out

df_out
'''

In [ ]:
'''
filter= df["spmode"]== "-999"

g= sns.PairGrid(df[filter].sample(500), vars= ["BpostFitMes","BpostFitDeltaE"], hue= "spmode")
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
'''

In [ ]:
'''
filter = df['spmode'] != '-999'
columns= df.columns

#g = sns.PairGrid(df[filter].sample(500), vars=['BpostFitMes', 'BpostFitDeltaE'], hue='spmode')
g = sns.PairGrid(df[filter].sample(50), vars=columns[1:6], hue='spmode')

g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend()
'''

In [ ]:
'''
feature_names= columns[1:] ##exclude spmode
print(feature_names)
'''

In [ ]:
'''
dcuts= bat.get_final_masks(data, region_definitions= region_definitions)

for key,val in dcuts.items():
    print(key, val['name'])
'''

In [ ]:
'''
cut1 = dcuts[1]['event']
print(len(cut1))

len(dcuts[3]['event'][cut1])
'''

# Generate the cuts

In [ ]:

##################################################
# SP
dcuts= bat.get_final_masks(data, region_definitions= region_definitions)



###################################
# Collision
dcuts_col= bat.get_final_masks(data_collision, region_definitions= region_definitions)



## Can start here when re-running a training session

In [ ]:
subset = ['spmode', 'BpostFitMes', 'BpostFitDeltaE', 'Lambda0_unc_Mass', \
      'BtagSideMes', 'BSphr', 'BThrust', 'BCosThetaS', \
      'R2', 'R2All', \
      'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
      'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
      'BThrustROE', 'BcosthCM']


In [ ]:
###################################
# SP
mask_event= dcuts[1]["event"]# & dcuts[2]["event"] & dcuts[3]["event"] 
mask= mask_event

df_sp = bat.dump_awkward_to_dataframe(data[mask], fields_to_dump=subset)#, dropna=True)

# Put the cuts into the dataframe 
cut1 = dcuts[1]['event'] # This is the main cut that gets rid of duplicates
cuts_to_add = [2, 3, 4, 6, -1]
for cut in cuts_to_add:
    bools = dcuts[cut]['event']
    colname = f'cut_{cut}'
    print(colname, len(bools[cut1]), bools[cut1])

    df_sp[colname] = bools[cut1]

# Drop entries where there are nans
print(len(df_sp))
df_sp.dropna(inplace=True)
print(len(df_sp))


In [ ]:
###################################################
# Collision
mask_event= dcuts_col[1]["event"]# & dcuts[2]["event"] & dcuts[3]["event"] 
mask= mask_event

df_col = bat.dump_awkward_to_dataframe(data_collision[mask], fields_to_dump=subset)

# Put the cuts into the dataframe 
cut1 = dcuts_col[1]['event'] # This is the main cut that gets rid of duplicates
cuts_to_add = [2, 3, 4, 6, -1]
for cut in cuts_to_add:
    bools = dcuts_col[cut]['event']
    colname = f'cut_{cut}'
    print(colname, len(bools[cut1]), bools[cut1])

    df_col[colname] = bools[cut1]

# Drop entries where there are nans
print(len(df_col))
df_col.dropna(inplace=True)
print(len(df_col))


df_sp

In [ ]:
df_col

In [ ]:
df_sp['spmode'].value_counts()

In [ ]:
mask = (df_sp['cut_2']==True) & (df_sp['cut_3']==True)

df_sp['spmode'][mask].value_counts()

In [ ]:
#df_sp.iloc[1287624]


In [ ]:
mask = df_sp['cut_-1'] == True
df_sp[mask].hist('BpostFitMes', bins=100)

In [ ]:
###################################################
# Test training routine on a small subset
###################################################

'''
mask = (df_sp['cut_2']==True) & (df_sp['cut_3']==True)

df_temp = df_sp[mask]

columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                   "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                  'BSphr', 'BThrust']
#["spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass","BtagSideMes"]


#model_filename = "MODEL_MLPClassifier_CUTS_1_2_3_nsig_2000_nbkg_2000_BNC.pkl"
model_filename = f"MODEL_MLPClassifier_CUTS_1_2_3_nsig_2000_nbkg_2000{BNC_tag}.pkl"

workspace = model_maker(df_temp, columns_to_drop=columns_to_drop, \
                                                      n_sig_bkg=[2000, 2000], model_filename=model_filename)#, 

model = workspace['model']
x_train = workspace['x_train']
y_train = workspace['y_train']
x_test = workspace['x_test']
y_test = workspace['y_test']

model
''';

In [ ]:
# Uncomment to plot the variables
'''
# Make a temporary dataframe with the cuts

mask = (df_sp['cut_2']==True) & (df_sp['cut_3']==True)

df_temp = df_sp[mask]

sp_mask = (df_temp['spmode']=='-999') | (df_temp['spmode']=='998')

plot_training_variables(df_temp[sp_mask])
'''
;

In [ ]:
#fig,ax = plt.subplots(figsize=(16,16))
#mask = df_temp['spmode'] == '998'
##corr = df_temp[mask].drop(columns=['spmode']).corr()
##corr.style.background_gradient(cmap='coolwarm', axis=None).format(precision=2)

#sns.heatmap(df_temp[mask].drop(columns=['spmode']).corr(), center=0, cmap='coolwarm', annot=True, fmt='.2f', annot_kws={"size": 8})

In [ ]:
#mask = df_sp['spmode'] == '-999'
#corr = df_sp[mask].drop(columns=['spmode']).corr()
#corr.style.background_gradient(cmap='coolwarm', axis=None).format(precision=2)

In [ ]:
df_sp.columns

In [ ]:
'''
columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                   "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                  'BSphr', 'BThrust']
#["spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass","BtagSideMes"]

model_filename = "TEST_MODEL_SAVE.pkl"
workspace = model_maker(df_temp, columns_to_drop=columns_to_drop, \
                                                      n_sig_bkg=[2000, 2000], model_filename=model_filename)#, 

model = workspace['model']
x_train = workspace['x_train']
y_train = workspace['y_train']
x_test = workspace['x_test']
y_test = workspace['y_test']

model
'''


mask = (df_sp['cut_2']==True) & (df_sp['cut_3']==True)

df_temp = df_sp[mask]

# Features 1
'''
columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                   "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                  'BSphr', 'BThrust']
'''

# Features 2
'''
columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                   "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                  'BSphr', 'BThrust', 'R2', 'R2All', 'sphericityAll', 'BtagSideMes']
'''

# Features 3
'''
columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                   "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                  'BSphr', 'BThrust', 'R2',  'sphericityAll', 'BtagSideMes', \
       'thrustMag', 'sphericityAll', 'BCosSphr', 'BCosThrust', \
                   'thrustCosTh', 'thrustCosThAll', \
       'BR2ROE', \
                  ]
'''

# Features 4
'''
columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                   "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                  'BSphr', 'BThrust', 'R2',  'sphericityAll', 'BtagSideMes', \
                  'thrustMag', 'sphericityAll', 'BCosSphr', 'BCosThrust', \
                   'thrustCosTh', 'thrustCosThAll', 'BSphrROE', \
                   'BThrustROE',  'BR2ROE', \
                  ]
'''

# Features 5
#'''
columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                   "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                  'BSphr', 'BThrust', 'R2',  'sphericityAll', 'BtagSideMes', \
                  'thrustMag', 'sphericityAll', 'BCosSphr', 'BCosThrust', \
                   'thrustCosTh', 'thrustCosThAll', 'BSphrROE', \
                   'BThrustROE',  'BR2ROE', 'R2All', 'thrustMagAll'\
                  ]
#'''


#["spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass","BtagSideMes"]

#model_filename = "TEST_MODEL_SAVE.pkl"
#model_filename = "MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000_BNC.pkl"

ntrain_sig = 30000
ntrain_bkg = 30000
trial = 4

ntrain_tag = f'features_5_nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'
#ntrain_tag = f'features_4_nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'
#ntrain_tag = f'features_3_nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'
#ntrain_tag = f'features_2_nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'
#ntrain_tag = f'nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'

model_filename = f"MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}_{BNC_tag}.pkl"

workspace = model_maker(df_temp, columns_to_drop=columns_to_drop, 
                        test_size_pct= 0.3, \
                        n_sig_bkg=[ntrain_sig, ntrain_bkg],\
                        model_filename=model_filename)#, 

print(f"Saving to {model_filename}")

model = workspace['model']
x_train = workspace['x_train']
y_train = workspace['y_train']
x_test = workspace['x_test']
y_test = workspace['y_test']

model

In [ ]:
model.feature_names

In [ ]:
workspace = joblib.load(model_filename)
#workspace

model = workspace['model']

model

In [ ]:
workspace

### Not done yet 

In [ ]:
x = np.array([10, 20, 30, 40])
np.random.shuffle(x)

x

In [ ]:
#workspace = joblib.load(model_filename)

#model_training_quality(workspace)

In [ ]:
#df_sp.loc(1287624)

In [ ]:
'''
#workspace['
idx_sig_train = workspace['idx_sig_train']

print(len(df_sp))

print(max(idx_sig_train))

#df_sp.loc[idx_sig_train, "used_in_sig_train"] = True

print(len(df_sp.index), len(idx_sig_train))
print()
print()


for idx in idx_sig_train[0:1000]:
    print(idx)
    print(df_sp.loc[idx]['spmode'])

''';

In [ ]:
#df_sp.iloc[1286295]

In [ ]:
'''
x = df_col.columns

print(len(x),x)
idx = x.get_loc('proba')

x  = x.delete(idx)

print(len(x),x)

boolvals = x.isin(['prob']).astype(int)

print(sum(boolvals))
'''

In [ ]:
#r = np.random.random(10)

    

In [ ]:
workspace.keys()

In [ ]:
#df_col.dropna(inplace=True)
add_probas_to_dfs(workspace, df_col, df_sp)#.drop(columns=columns_to_drop))


df_col#.columns

In [ ]:
df_sp

In [ ]:
outfilename = f"DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
df_sp.to_parquet(outfilename)

outfilename = f"DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
df_col.to_parquet(outfilename)

In [ ]:
mask = df_col['cut_-1']==True

df_col[mask]['proba'].hist(bins=25, range=(0,0.99))
#plt.yscale('log')




In [ ]:
print(workspace.keys())
workspace['idx_sig_not_train']

In [ ]:
#for i in range(0,3):

#idx = workspace['idx_sig_not_train']
idx = workspace['idx_bkg_not_train']

print(len(idx), len(df_sp))

print(type(idx),idx)

if len(idx)>0:
    print(max(idx))

print(df_sp.index)
print(max(df_sp.index))
#df_sp.loc[100240]
df_tmp = df_sp.loc[idx]

#for i in idx:

#    print(f"{i} {df_sp.loc[i].index}")

#'''
#spmask = (df_tmp['spmode']=='998')
#spmask = (df_tmp['spmode']=='-999')
spmask = (df_tmp['spmode']!='-999')

mask =   (df_tmp['cut_2']==True) & (df_tmp['cut_3']==True)  & (df_tmp['cut_4']==True)
mask = mask & (df_tmp['proba'] > 0.97)

#var = 'proba'
var = 'BpostFitMes'
#df_tmp[var].hist(bins=25)#, range=(0,0.99))
df_tmp[spmask & mask][var].hist(bins=25)#, range=(0,0.99))
#plt.yscale('log')

print(df_tmp[spmask & mask]['spmode'])
print(idx[0:10])
#df_sp[spmask & mask]['proba'].loc[idx[0:10]]#, range=(0,0.99))

print(df_sp.index)
#'''
;
df_tmp

In [ ]:
model = workspace['model']

x_train = workspace['x_train']
y_train = workspace['y_train']
y_test = workspace['y_test']

#x_train
#y_train
y_test

In [ ]:
#model.feature_names = ['hi']


In [ ]:
#model.feature_names

In [ ]:
dcuts.keys()

for key in dcuts.keys():
    print(f"key: {key:3d}   name: {dcuts[key]['name']}")

In [ ]:
#def punzi_fom_nn(model_aft_train, sp_data, threshold, sp_998_df, sp_999_df, sig_disc= 4, scaling= 0.3):
def punzi_fom_nn(df_sp, df_col, sig_sp_mode='-999', region_definitions = None, sigma = 4.0, BNC=False):

    # Collision data
    mask = (df_col['cut_-1'] == True) 
    if BNC is True:
        mask = (df_col['cut_2'] == True) 
        mask = mask & (df_col['cut_3'] == True) 
        mask = mask & (df_col['cut_4'] == True) 
        
    
    df_col_tmp = df_col[mask]

    # SP
    mask = (df_sp['cut_-1'] == True) 
    if BNC is True:
        mask = (df_sp['cut_2'] == True) 
        mask = mask & (df_sp['cut_3'] == True) 
        mask = mask & (df_sp['cut_4'] == True) 

    mask = mask & (df_sp['spmode'] == sig_sp_mode)
    mask = mask & (df_sp['used_in_sig_train'] == False)
    df_sp_tmp = df_sp[mask]

    meslo = region_definitions['signal MES'][0]
    meshi = region_definitions['signal MES'][1]
    
    delo = region_definitions['signal DeltaE'][0]
    dehi = region_definitions['signal DeltaE'][1]

    messidelo = region_definitions['sideband MES'][0]
    messidehi = region_definitions['sideband MES'][1]
    
    desidelo1 = region_definitions['sideband 1 DeltaE'][0]
    desidehi1 = region_definitions['sideband 1 DeltaE'][1]
    
    desidelo2 = region_definitions['sideband 2 DeltaE'][0]
    desidehi2 = region_definitions['sideband 2 DeltaE'][1]

    
    fom_dict = {}
    fom_dict['thresh'] = []
    fom_dict['nbkg_sb1'] = []
    fom_dict['nbkg_sb2'] = []
    fom_dict['nbkg'] = []
    fom_dict['nsig'] = []

    # Collision data
    mes_col = df_col_tmp['BpostFitMes']
    de_col = df_col_tmp['BpostFitDeltaE']

    mask1_col = (mes_col>messidelo) & (mes_col<messidehi) & (de_col>desidelo1) & (de_col<desidehi1)    
    mask2_col = (mes_col>messidelo) & (mes_col<messidehi) & (de_col>desidelo2) & (de_col<desidehi2)

    # SP
    mes_sp = df_sp_tmp['BpostFitMes']
    de_sp = df_sp_tmp['BpostFitDeltaE']

    mask_sp = (mes_sp>meslo) & (mes_sp<meshi) & (de_sp>delo) & (de_sp<dehi)


    for thresh in np.arange(0,1,0.01):
        
        # Collision data
        mask_thresh_col = df_col_tmp['proba'] > thresh

        nsb1 = len(df_col_tmp[mask1_col & mask_thresh_col])        
        nsb2 = len(df_col_tmp[mask2_col & mask_thresh_col])
    
        # Collision data
        mask_thresh_sp = df_sp_tmp['proba'] > thresh

        nsig = len(df_sp_tmp[mask_sp & mask_thresh_sp])        
    
        #print(nsb1, nsb2, nsig)
        
        fom_dict['thresh'].append(thresh)
        fom_dict['nbkg_sb1'].append(nsb1)
        fom_dict['nbkg_sb2'].append(nsb2)
        #fom_dict['nbkg'].append((nsb1 + nsb2)/2)
        fom_dict['nbkg'].append(nsb1 + nsb2)
        fom_dict['nsig'].append(nsig)

    df_fom = pd.DataFrame.from_dict(fom_dict)
    df_fom['sig_pct'] = df_fom['nsig'] / df_fom['nsig'].iloc[0]

    sigma = 4.0
    
    df_fom['fom'] = df_fom['sig_pct'] / (np.sqrt(df_fom['nbkg']) + sigma/2.0)

    return df_fom

In [ ]:
df_fom = punzi_fom_nn(df_sp, df_col, region_definitions=region_definitions, BNC=BNC_bool)


In [ ]:
df_fom.plot(x='thresh', y='fom')
plt.ylabel("FOM")
plt.xlabel("threshold")


df_fom.plot(x='thresh',y='sig_pct')
plt.ylabel("$\%$ signal retained")
plt.xlabel("threshold")


df_fom.plot(x='thresh',y='nbkg')
plt.ylabel("n bkg events")
plt.xlabel("threshold")


In [ ]:
fom_max = df_fom['fom'].max()

print(fom_max)

filter = df_fom['fom'] == fom_max

df_fom[filter]

In [ ]:
max_cut = df_fom[filter]['thresh'].values[0]
print(f'max_cut: {max_cut}')

In [ ]:
df_fom.sort_values(by='fom')[-20:]

In [ ]:
probacut =max_cut
#probacut = 0.90

plt.figure(figsize=(8,8))

for i in range(0,3):

    idx = None
    spmode = None
    df_tmp = None
    
    if i==0:
        idx = workspace['idx_bkg_not_train']
        spmode = '998'
        df_tmp = df_sp.loc[idx]

    elif i==1:
        idx = workspace['idx_sig_not_train']
        spmode = '-999'
        df_tmp = df_sp.loc[idx]
    
    elif i==2:
        spmode = '0'
        df_tmp = df_col
    
    spmask = (df_tmp['spmode']==spmode)

    mask =   (df_tmp['cut_-1']==True)
    if BNC_bool:
        mask =   (df_tmp['cut_2']==True) & (df_tmp['cut_3']==True)  & (df_tmp['cut_4']==True)

    mask = mask & (df_tmp['proba'] > probacut)
    
    mask = mask & (df_tmp['BpostFitDeltaE']<0.07) & (df_tmp['BpostFitDeltaE']>-0.07)

    if BNC_bool:
        mask = mask & (df_tmp['BpostFitDeltaE']<0.05) & (df_tmp['BpostFitDeltaE']>-0.05)

    
    #var = 'proba'
    var = 'BpostFitMes'

    plt.subplot(3,1,i+1)
    df_tmp[spmask & mask][var].hist(bins=50, range=(5.2,5.3))#, range=(0,0.99))


In [ ]:
df_sp[['BpostFitMes', 'proba']].corr()

In [ ]:
mask = (df_sp['spmode'] == '998')
mask = mask &  (df_sp['cut_2']==True) & (df_sp['cut_3']==True)  & (df_sp['cut_4']==True)
mask = mask & (df_sp['BpostFitMes']>5.20)# & (df_sp['BpostFitDeltaE']>-0.07)


mask = mask & (df_sp['BpostFitDeltaE']<0.07) & (df_sp['BpostFitDeltaE']>-0.07)

if BNC_bool:
    mask = mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)


mask = mask & (df_sp['proba'] > max_cut)


df_sp[mask].plot.scatter(x='BpostFitMes', y='proba')

In [ ]:
df_sp[mask][['BpostFitMes', 'proba']].corr()

In [ ]:
mask = (df_sp['spmode'] == '-999')
mask = mask &  (df_sp['cut_2']==True) & (df_sp['cut_3']==True)  & (df_sp['cut_4']==True)
mask = mask & (df_sp['BpostFitMes']>5.20)# & (df_sp['BpostFitDeltaE']>-0.07)


mask = mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)

df_sp[mask].hist('BpostFitMes', bins=500, range=(5.27,5.271))


# Run many

In [ ]:
mask = (df_sp['cut_2']==True) & (df_sp['cut_3']==True)

df_sp[mask]['spmode'].value_counts()

In [ ]:
def run_a_full_trial(data, data_collision, features_to_drop=1, ntrain_sig=60000, ntrain_bkg=60000, trial=1):
    subset = ['spmode', 'BpostFitMes', 'BpostFitDeltaE', 'Lambda0_unc_Mass', \
      'BtagSideMes', 'BSphr', 'BThrust', 'BCosThetaS', \
      'R2', 'R2All', \
      'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
      'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
      'BThrustROE', 'BcosthCM']

    ###################################
    # SP
    mask_event= dcuts[1]["event"]
    mask= mask_event
    
    df_sp = bat.dump_awkward_to_dataframe(data[mask], fields_to_dump=subset)#, dropna=True)
    
    # Put the cuts into the dataframe 
    cut1 = dcuts[1]['event'] # This is the main cut that gets rid of duplicates
    cuts_to_add = [2, 3, 4, 6, -1]
    for cut in cuts_to_add:
        bools = dcuts[cut]['event']
        colname = f'cut_{cut}'
        print(colname, len(bools[cut1]), bools[cut1])
    
        df_sp[colname] = bools[cut1]
    
    # Drop entries where there are nans
    print(len(df_sp))
    df_sp.dropna(inplace=True)
    print(len(df_sp))

    ###################################################
    # Collision
    mask_event= dcuts_col[1]["event"]# & dcuts[2]["event"] & dcuts[3]["event"] 
    mask= mask_event
    
    df_col = bat.dump_awkward_to_dataframe(data_collision[mask], fields_to_dump=subset)
    
    # Put the cuts into the dataframe 
    cut1 = dcuts_col[1]['event'] # This is the main cut that gets rid of duplicates
    cuts_to_add = [2, 3, 4, 6, -1]
    for cut in cuts_to_add:
        bools = dcuts_col[cut]['event']
        colname = f'cut_{cut}'
        print(colname, len(bools[cut1]), bools[cut1])
    
        df_col[colname] = bools[cut1]
    
    # Drop entries where there are nans
    print(len(df_col))
    df_col.dropna(inplace=True)
    print(len(df_col))
    
    #df_sp

    
    mask = (df_sp['cut_2']==True) & (df_sp['cut_3']==True)
    
    df_temp = df_sp[mask]

    columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1']
    if features_to_drop==1:
        # Features 1
        columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                           "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                          'BSphr', 'BThrust']

    elif features_to_drop==2:
        # Features 2
        columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                           "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                          'BSphr', 'BThrust', 'R2', 'R2All', 'sphericityAll', 'BtagSideMes']

    elif features_to_drop==3:
        # Features 3
        columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                           "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                          'BSphr', 'BThrust', 'R2',  'sphericityAll', 'BtagSideMes', \
               'thrustMag', 'sphericityAll', 'BCosSphr', 'BCosThrust', \
                           'thrustCosTh', 'thrustCosThAll', \
               'BR2ROE', \
                          ]

    elif features_to_drop==4:
        # Features 4
        columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                           "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                          'BSphr', 'BThrust', 'R2',  'sphericityAll', 'BtagSideMes', \
                          'thrustMag', 'sphericityAll', 'BCosSphr', 'BCosThrust', \
                           'thrustCosTh', 'thrustCosThAll', 'BSphrROE', \
                           'BThrustROE',  'BR2ROE', \
                          ]
    elif features_to_drop==5:
        # Features 5
        columns_to_drop = ['cut_2', 'cut_3', 'cut_4', 'cut_6', 'cut_-1', \
                           "spmode","BpostFitMes","BpostFitDeltaE","Lambda0_unc_Mass", \
                          'BSphr', 'BThrust', 'R2',  'sphericityAll', 'BtagSideMes', \
                          'thrustMag', 'sphericityAll', 'BCosSphr', 'BCosThrust', \
                           'thrustCosTh', 'thrustCosThAll', 'BSphrROE', \
                           'BThrustROE',  'BR2ROE', 'R2All', 'thrustMagAll'\
                      ]

    #ntrain_sig = 60000
    #ntrain_bkg = 60000
    #trial = 4
    
    ntrain_tag = f'features_{features_to_drop}_nsig_{ntrain_sig}_nbkg_{ntrain_bkg}_trial{trial:0d}'
    
    model_filename = f"MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}_{BNC_tag}.pkl"
    
    workspace = model_maker(df_temp, columns_to_drop=columns_to_drop, 
                            test_size_pct= 0.3, \
                            n_sig_bkg=[ntrain_sig, ntrain_bkg],\
                            model_filename=model_filename)#, 
    
    print(f"Saving to {model_filename}")
    
    model = workspace['model']
    x_train = workspace['x_train']
    y_train = workspace['y_train']
    x_test = workspace['x_test']
    y_test = workspace['y_test']
    
    #model
    add_probas_to_dfs(workspace, df_col, df_sp)#.drop(columns=columns_to_drop))
    
    outfilename = f"DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
    df_sp.to_parquet(outfilename)
    
    outfilename = f"DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
    df_col.to_parquet(outfilename)

    return model, df_col, df_sp

In [ ]:
for i in range(1,6):
    print(f"--------------------------------------------------\ntrial {i}\n-------------------------------------")
    #model, df_col, df_sp = run_a_full_trial(data, data_collision, ntrain_sig=60000, ntrain_bkg=60000, features_to_drop=5, trial=i)
    model, df_col, df_sp = run_a_full_trial(data, data_collision, ntrain_sig=30000, ntrain_bkg=30000, features_to_drop=2, trial=i)

In [ ]:
df_col